<a href="https://colab.research.google.com/github/hsieh2000/HW/blob/main/foreign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#匯入套件
import time 
import datetime
from dateutil.relativedelta import relativedelta
import json 
import math
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from fake_useragent import UserAgent
import random
import warnings
import os

#讀取config.json
config_path = "../config.json" #預設路徑
with open(config_path) as f:   #開啟config.json
    config = json.load(f)      #以json形式讀取config.json
    root = config["root"]      #取得檔案輸出根目錄


# 建立資料夾
newest_path = f'{root}最新檔/'                  #設定最新檔匯出路徑
log_path = f'{root}境外基金基本資訊記錄檔/'     #設定紀錄檔匯出路徑
dir = ["csv/", "txt/", "err/"]                #設定檔案存放資料夾清單

for i in dir:                                 #以for迴圈方式建立資料夾
  if not os.path.isdir(log_path+i):            #if判斷資料夾是否已經存在
    os.makedirs(log_path+i)                   #如果不存在就新增

if not os.path.isdir(newest_path):            #if判斷資料夾是否已經存在
    os.mkdir(newest_path)                     #如果不存在就新增

# 路徑設定
filename = "境外基金基本資訊"                   #設定輸出檔名                 
bug_url = "境外基金錯誤連結"		            #設定錯誤連結檔名
csv_path = log_path+dir[0]                     #設定csv檔輸出路徑 
txt_path = log_path+dir[1]                    #設定txt檔輸出路徑 
err_path = log_path+dir[2]        	            #設定錯誤連結輸出路徑 

# 參數設定
delay = random.uniform(1, 5)                  #設定隨機數1至5
warnings.filterwarnings("ignore")             #忽略警告訊息
user_agent = UserAgent()                      #調用 UserAgent() 使用者代理

header = {"user-agent" : user_agent.random,
      "Referer" : "https://announce.fundclear.com.tw/MOPSFundWeb/E03_01.jsp"}  #設定爬蟲header

link_header = 'https://announce.fundclear.com.tw/MOPSFundWeb/'                 #設定連結表頭(可刪

# 資料月份判斷
if int(datetime.datetime.now().strftime("%d"))>10:                             #if判斷本月日期是否大於10號
    month = datetime.datetime.now() - relativedelta(months=1)                  #為真則將當前月份-1存入month變數
    month = month.strftime("%Y%m")                                             #將month變數轉換為"西元年月"格式的字串
else:                                                                          #else
    month = datetime.datetime.now() - relativedelta(months=2)                  #為假則將當前月份-2存入month變數
    month = month.strftime("%Y%m")                                             #將month變數轉換為"西元年月"格式的字串



class newForeignFund(object):                                                     #定義名為newForeignFund的類別

    def __init__(self):                                                           #宣告 def __init__()
        with open(config_path) as f:                                              #開啟config.json檔
            config = json.load(f)                                                 #以json形式讀取config.json
            target_agent_list = config['agent']                                   #讀取config.json中的agent中的目標機構代碼                                         
            print(target_agent_list)                                              #印出目標投信機構
            with open(f'{txt_path}{filename}{month}.txt', mode='w') as T:         #在txt的資料夾下新增一個檔名為"境外基金基本資訊+西元年月"的txt檔，模式為寫入
                T.write('{\'funds\':[')                                           #寫入'{\'funds\':['以符合json格式

        a = self.general_agent()                                                  #執行general_agent() 並將結果存入變數a
        a = self.compare(a,target_agent_list)                                     #執行compare() 並將結果存入變數a
        a = self.tab_link_generaltor(a)                                           #執行tab_link_generaltor() 並將結果存入變數a
        a = self.meta_link_generator(a)                                           #執行meta_link_generator() 並將結果存入變數a
        a = self.meta_crawler(a)                                                  #執行meta_crawler() 並將結果存入變數a
        a = self.toDataframe(a)                                                   #執行toDataframe() 並將結果存入變數a
        a = self.store(a)                                                         #執行store() 並將結果存入變數a


    def general_agent(self):                                                                                                #宣告 general_agent()
        url = 'https://announce.fundclear.com.tw/MOPSFundWeb/O01.jsp?organizType=ALL&organizCom=ALL&fundCom=ALL'            #將基金觀測站的機構列表頁網址存入變數url
        header = {"user-agent" : user_agent.random}                                                                         ####可刪
        res=requests.get(url,headers = header, timeout=30)                                                                  #取得網頁原始內容，並設定請求時間上限為30秒
        soup = BeautifulSoup(res.text, 'html.parser')                                                                       #透過BeautifulSoup解析原始內容，解析模式為"html.parser"
        webText = str(soup.select("select[name='organizCom'] option"))                                                      #選取name='organizCom'的所有option，並將結果轉換為字串存入webText變數
        regex = re.compile('\w\d{4}">\w+')                                                                                  #定義正規表達式'\w\d{4}">\w+'並存入regex變數，目的為過濾出所有機構名稱與代碼
        result = re.findall(regex, webText)                                                                                 #透過re.findall找出符合regex規則的所有結果並存入result變數
        result = result[2:]                                                                                                 #由於result的前兩項固定為不相干內容，因此去除想兩項結果
        agent_name = list(map(lambda x : x.split('">')[1],result))                                                          #對result內元素以">"為中心進行拆分並選擇後半段作為機構名稱，將結果存回agent_name
        agent_ID = list(map(lambda x : x.split('">')[0],result))                                                            #對result內元素以">"為中心進行拆分並選擇前半段作為機構名稱，將結果存回agent_ID
        print(agent_name)                                                                                                   #印出全數機構名稱
        return agent_ID                                                                                                     #返回agent_ID

    def compare(self, agent_ID, target_list):                                                                               #宣告 compare()，此function需要輸入兩個參數，分別為全部機構代碼列表與目標機構代碼列表
        # target_list = ['A0038','A0031','A0018','B0029','A0036','A0011','A0032']
        target_Id = list(set(target_list).intersection(set(agent_ID)))                                                      #取得全部機構代碼列表與目標機構代碼列表的交集，並將結果存入target_Id變數
        print(target_Id)                                                                                                    #印出有效的目標機構代碼列表
        return target_Id                                                                                                    #返回有效的目標機構代碼列表

    def tab_link_generaltor(self, target_Id):                                                                               #宣告 tab_link_generaltor()，此function需要輸入一個變數，為有效目標機構代碼列表
        regex2 = re.compile("A01_01.jsp?.+establishYear=ALL")                                                               #定義正規表達式'A01_01.jsp?.+establishYear=ALL'並存入regex2變數，目的為生成各機構的銷售基金總覽網址
        link_header = 'https://announce.fundclear.com.tw/MOPSFundWeb/'                                                      #設定連結表頭
        target_link_list=[]                                                                                                 #宣告一個名為target_link_list的空list

        for ID in target_Id:                                                                                                                                #透過for迴圈將機構ID帶入網址模板，取得各機構的基金總覽網址       
            agent_url = f'https://announce.fundclear.com.tw/MOPSFundWeb/O01.jsp?organizType=ALL&organizCom={ID}&fundCom=ALL'
            res=requests.get(agent_url,headers = header, timeout=10)
            soup = BeautifulSoup(res.text, 'html.parser')
            AgentWebText = soup.select('td[style="cursor: hand; color: blue"]')
            link = [ regex2.search(str(s)).group(0) for s in AgentWebText if regex2.search(str(s)) ]
            link = list(map(lambda x : link_header+x.replace(';','&').replace('®','&reg').replace('establishYear=ALL','establishYear=&pid='),link))
            target_link_list = target_link_list + link                                                                                                      #將生成的網址存入target_link_list
            time.sleep(delay)                                                                                                                               #隨機休眠1~5秒
        return target_link_list                                                                                                                             #返回目標基金總覽網址清單

    def retry_page(self, url):                                                                                                                              #宣告 retry_page()，此function需要輸入一個參數，為網址
        times = 0                     															                                                            #宣告times變數為0
        while times<3:                                                                                                                                      #透過while迴圈，當取得網頁內容出錯時會重新嘗試取得內容，最多嘗試三次                                                     
            try:
                res=requests.get(url,headers = header)
                soup = BeautifulSoup(res.text, 'html.parser')
                return soup
            except:
                times+=1
                print(f"try {times} time, {url}")
                pass


    #這三個def架構為 meta_link_generator() >> changing_page() >> ID_crawler()
    def ID_crawler(self, url, ID_link):						            				                                                                    #宣告 ID_crawler()，此function需要輸入兩個參數，分別為網址與目標機構代碼列表
        main = 'main1.jsp?fundId='											
        soup = self.retry_page(url)											                                                                                #嘗試取得網頁內容
        fundID = soup.select('tr.row1 td[style="cursor: hand;"]')+ soup.select('tr.row2 td[style="cursor: hand;"]')                                         #取得包含各檔基金代碼的元素
        regex4 = re.compile("A01_02.jsp\?fundId=.+',")									                                                                    #定義正規表達式A01_02.jsp\?fundId=.+',並存入regex4變數，目的為生成基金基本資訊網址
        link2 = [regex4.search(str(s)).group(0) for s in fundID if regex4.search(str(s))]				                                                    #篩選出符合regex4的元素，並將結果存入link2
        _ID_link = list(map(lambda x : link_header+main+x.replace("',","").split('=')[1],link2))			                                                #_ID_link為當前頁面各檔基金的基本資訊頁面連結
        ID_link = ID_link+_ID_link											
        TdRight = soup.select('td[align="right"]')
        regex3 = re.compile("(.*)")    
        summary = regex3.search(str(TdRight)).group(0)
        try:
            full_page = int(re.search('\d{1,2}',summary.split(',')[0]).group(0))					                                                        #full_page為此機構基金列表的總頁數，無其他分頁則讓#full_page等於1
        except:
            full_page = 1
        time.sleep(delay)												                                                                                    #隨機休眠1~5秒
        return (url, full_page, ID_link)										                                                                            #返回當前頁面網址、總頁數、基金基本資訊網址清單

    def changing_page(self, firsturl, ID_link):								    #宣告 changing_page()，此function需要輸入兩個參數，分別為機構基金列表的第一頁網址與基金基本資訊網址清單
        i=1														                #定義變數i為1
        url = firsturl+str(i)												    #爬蟲網址為機構基金列表的第一頁網址加上i
        print(url)													            #列印當前網址
        temp = self.ID_crawler(url,ID_link)										#將 ID_crawler() 的結果存入變數temp
        ID_link = temp[2]												
        end_page = temp[1]
        url = temp[0]
        print(f'page1/{end_page}')

        for j in range(2,int(end_page)+1):										#透過 for 迴圈實現爬蟲可自動換頁爬取直到最後一頁
            url = url.split('pid=',1)[0]+'pid='+str(j)
            print(url)

            sub_temp = self.ID_crawler(url,ID_link)
            ID_link=sub_temp[2]
            print(f'page{j}/{end_page}')										#列印出目前爬蟲進度
            time.sleep(delay)												    #隨機休眠1~5秒
        return ID_link													        #返回基金基本資訊網址清單

    def meta_link_generator(self, target_link_list):							#宣告 meta_link_generator()，此function需要輸入一個變數，為目標基金總覽網址清單
        a = []
        ID_link = []													        #定義變數ID_link為空list
        for x,y in enumerate(target_link_list):									#透過 for 迴圈達成針對目標機構爬取基金資訊
            a = a + self.changing_page(y, ID_link)
            print(f'{x+1}/{len(target_link_list)}')								#列印出目前爬蟲頁數
        return a													            #返回全部基金基本資訊網址


    def Replace(self, x):												                                        #宣告 Replace()，此function需要輸入一個變數，為經過BeautifulSoup解析過的網頁元素
        x = x.text													                                            #篩選元素中的純文字訊息
        x = x.replace('\t','').replace('\n','').replace('\xa0','').replace('\r','').replace('\"','')			#將空白字元移除
        return x

    def ToJSON(self, keys, values):												#宣告 ToJSON()，此function需要輸入兩個參數，分別為key值的list與value值的list
        a = list(map(lambda x : self.Replace(x), keys))                         
        b = list(map(lambda x : self.Replace(x), values))
        res = {a[i]: b[i] for i in range(len(keys))}
        json_object = json.loads(json.dumps(res, ensure_ascii=False))           #json.dumps回傳type為str，要用;json.loads()轉為dict
        return json_object

    def Convert(self, fund_name):                                               #宣告 Convert()，此function需要輸入一個變數，為基金名稱
        name = '基金名稱'
        res_dct = {name : fund_name}
        res_dct = json.loads(json.dumps(res_dct, ensure_ascii=False))           #json.dumps回傳type為str，要用;json.loads()轉為dict
        return res_dct

    def get_source(self, url):                                                  #宣告 get_source()，此function需要輸入一個變數，為網址
        times = 0
        while times<3 :                                                         #透過while迴圈，當取得網頁內容出錯時會重新嘗試取得內容，最多嘗試三次 
            try:
                res = requests.get(url, headers=header)
                soup = BeautifulSoup(res.text, 'html.parser')
                c_name = soup.select('font[color="#003399"]')[0]
                return (soup, c_name)
            except:
                times+=1
                print(f"try {times} time, {url}")
                pass


    def meta_crawler(self, link_list):                                          #宣告 meta_crawler()，此function需要輸入一個變數，為基金基本資訊網址的list
        with open(f'{err_path}{bug_url}.txt', mode='w') as D:                   #在err_path的資料夾下新增一個錯誤連結記錄檔，模式為寫入
            D.write("Error url:\n")
            D.close()
        json_list=[]
        for num, i in enumerate(link_list):                                     #透過 for 迴圈針對所有基金基本資訊連結進行資料爬取
            try:
                soup, c_name = self.get_source(i)
                c = self.Convert(self.Replace(c_name))
                title = soup.select('.FieldTitle')                              #篩選出細項標題
                content = soup.select('.FieldContent')                          #篩選出細項內容
                j = self.ToJSON(title, content)                                 
                c.update(j)                                                     
                with open(f'{txt_path}{filename}{month}.txt', mode='a') as T:   #將爬取資料寫入txt檔中
                    T.write(str(c))
                    if num+1 == len(link_list):
                      pass
                    else:
                      T.write(',')
                      
                json_list.append(c)
                time.sleep(delay)
            except:
                with open(f'{err_path}{bug_url}.txt', mode='a') as D:           #如果爬取失敗將失敗連結寫入錯誤連結記錄檔
                            D.write(f'{i},')
                            D.write("\n")
                            D.close() 
                print(f"{i} got problem")
                pass      
            #測試
            if num ==10:
              break

            print(f'{self.Replace(c_name)}, {num+1}/{len(link_list)}')
        with open(f'{txt_path}{filename}{month}.txt', mode='a') as T:
            T.write(']}')
            T.close()       
        return json_list

    def toDataframe(self, json_list):                                           #宣告 toDataframe()，此function需要輸入一個變數，為各檔基金資料匯集成的list
        df = pd.DataFrame(json_list,columns=list(json_list[0].keys()))          #將list of json 的資料轉換為pandas的dataframe
        return df

    def store(self, df):                                                         #宣告 store()，此function需要輸入一個變數，為dataframe
        df.to_csv(f'{csv_path}{filename}{month}.csv', encoding='utf-8')             #匯出csv
        df.to_csv(f'{newest_path}{filename}.csv', encoding='utf-8')                 #匯出csv

    # target_list = ['A0038','A0031','A0018','B0029','A0036','A0011','A0032']

if __name__ == '__main__':
    c = newForeignFund()


